In [ ]:
# Import packages
import os
from matplotlib import pyplot as plt
import pandas as pd

# Import AuTuMN modules
from autumn.settings import Models, Region
from autumn.settings.folders import OUTPUT_DATA_PATH, PROJECTS_PATH
from autumn.core.project import get_project
from autumn.core import db
from autumn.core.plots.calibration.plots import get_output_from_run_id
from autumn.core.plots.uncertainty.plots import _plot_uncertainty, _get_target_values
from autumn.core.plots.plotter.base_plotter import COLOR_THEME
from autumn.core.plots.utils import change_xaxis_to_date, REF_DATE, ALPHAS, COLORS, _apply_transparency, _plot_targets_to_axis

from autumn.calibration.utils import get_uncertainty_df
import matplotlib.patches as mpatches
import json

In [ ]:
# Specify model details
model = Models.SM_SIR
region = Region.NCR
date = "2022-07-19"
# assumption = "none"
dirname = date

include_scenario_descriptions = False

In [ ]:
# get the relevant project and output data
project = get_project(model, region)
project_calib_dir = os.path.join(
    OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name
)
calib_path = os.path.join(project_calib_dir, dirname)
# Load tables
mcmc_tables = db.load.load_mcmc_tables(calib_path)
mcmc_params = db.load.load_mcmc_params_tables(calib_path)

uncertainty_df = get_uncertainty_df(calib_path, mcmc_tables, project.plots)
scenario_list = uncertainty_df['scenario'].unique()

# make output directories
output_dir = dirname
base_dir = os.path.join("outputs", output_dir)
os.makedirs(base_dir, exist_ok=True)
dirs_to_make = ["calibration", "median", "uncertainty", "csv_files"]
for dir_to_make in dirs_to_make:
    os.makedirs(os.path.join(base_dir, dir_to_make), exist_ok=True)

In [ ]:
titles = {
    "notifications": "Daily number of notified COVID-19 cases",
    "infection_deaths":"Daily number of COVID-19 deaths", # "Cumulative deaths since 1 Mar 2021", # ,
    "accum_deaths": "Cumulative number of COVID-19 deaths",
    "accum_incidence": "Cumulative number of COVID-19 infections",
    "incidence": "Daily incidence (incl. asymptomatics and undetected)", # "Cumulative infections since 1 Mar 2021", # ,
    "hospital_occupancy": "Hospital beds occupied by COVID-19 patients",
    "icu_occupancy": "ICU beds occupied by COVID-19 patients",
    "hospital_admissions":"Hospital admissions", #"Cumulative hospitalisations since 1 Mar 2021", #
    "cdr": "Proportion detected among symptomatics",
    "proportion_vaccinated": "Proportion vaccinated",
    "prop_incidence_strain_delta": "Proportion of Delta variant in new cases",
    "cumulative_notifications": "Accumulated COVID-19 notifications",
    "cumulative_hospital_admissions": "Accumulated COVID-19 hospital admissions",
    "cumulative_icu_admissions": "Accumulated COVID-19 ICU admissions",
}

legend_titles = {
    0: "Baseline",
}
if include_scenario_descriptions:
    legend_titles.update(
        {sc: f"Sc. {sc}: {project.param_set.scenarios[sc-1].to_dict()['description']}" for sc in scenario_list if sc > 0}
    )
else:
    legend_titles.update(
        {sc: f"Sc. {sc}" for sc in scenario_list if sc > 0}
    )

def plot_outputs(output_type, output_name, scenario_list, sc_linestyles, sc_colors, show_v_lines=False, x_min=590, x_max=775, cumul_from=None):

    # plot options
    title = titles[output_name]
    title_fontsize = 18
    label_font_size = 15
    linewidth = 3
    n_xticks = 10

    # initialise figure
    fig = plt.figure(figsize=(12, 8))
    plt.style.use("ggplot")
    axis = fig.add_subplot()

    # prepare colors for ucnertainty
    n_scenarios_to_plot = len(scenario_list)
    uncertainty_colors = _apply_transparency(COLORS[:n_scenarios_to_plot], ALPHAS[:n_scenarios_to_plot])

    if output_type == "MLE":
        derived_output_tables = db.load.load_derived_output_tables(calib_path, column=output_name)
    for i, scenario in enumerate(scenario_list):    
        linestyle = sc_linestyles[scenario]
        color = sc_colors[scenario]
        zorder = 100 - scenario
        if output_type == "MLE":
            times, values = get_output_from_run_id(output_name, mcmc_tables, derived_output_tables, "MLE", scenario)
            if cumul_from is not None:
                ref_ind = list(times).index(cumul_from)
                cum_values = [0] * (ref_ind + 1) + [sum(values[ref_ind + 1: i]) for i in range(ref_ind + 1, len(values)) ] 
                values = cum_values
            
            axis.plot(times, values, color=color, linestyle=linestyle, linewidth=linewidth, zorder=zorder)
        elif output_type == "median":
            _plot_uncertainty(
                axis,
                uncertainty_df,
                output_name,
                scenario,
                x_max,
                x_min,
                [_, _, _, color],
                overlay_uncertainty=False,
                start_quantile=0,
                zorder=zorder,
                linestyle=linestyle,
                linewidth=linewidth,
             )
        elif output_type == "uncertainty":
            scenario_colors = uncertainty_colors[i]         
            _plot_uncertainty(
                axis,
                uncertainty_df,
                output_name,
                scenario,
                x_max,
                x_min,
                scenario_colors,
                overlay_uncertainty=True,
                start_quantile=0,
                zorder=scenario + 1,
             )
        else:
            print("Please use supported output_type option")


    axis.set_xlim((x_min, x_max))
    axis.set_title(title, fontsize=title_fontsize)
    plt.setp(axis.get_yticklabels(), fontsize=label_font_size)
    plt.setp(axis.get_xticklabels(), fontsize=label_font_size)
    change_xaxis_to_date(axis, REF_DATE)
    plt.locator_params(axis="x", nbins=n_xticks)

    if show_v_lines:
        release_dates = {624: "15 Sep 2021", 609: "31 Aug 2021"}
        y_max = plt.gca().get_ylim()[1]
        linestyles = ["dashdot", "solid"]
        i = 0
        for time, date in release_dates.items():
            plt.vlines(time, ymin=0, ymax=y_max, linestyle=linestyles[i])
            text = f"Lockdown relaxed on {date}"
            plt.text(time - 5, .5*y_max, text, rotation=90, fontsize=11)
            i += 1
            
    return axis


# Scenario plots with single lines

In [ ]:
scenario_x_min, scenario_x_max = 900, 1280    # 900 for 18 June 2022    640 for 1 Oct 2021

In [ ]:
output_names = ["notifications", "icu_occupancy", "hospital_occupancy", "cumulative_notifications", "cumulative_hospital_admissions", "cumulative_icu_admissions"]
#output_names = ["incidence", "infection_deaths", "hospital_admissions"]

requested_scenario_lists = {
    "all scenarios": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    "no new variant": [0, 2, 4, 6 ,8 , 10],
    "with new variant": [1, 3 , 5, 7, 9 ,11]
}


sc_colors = [COLOR_THEME[i] for i in scenario_list]
sc_linestyles = ["solid"] + ["solid"] * (len(scenario_list) - 1)

for foldername, requested_scenario_list in requested_scenario_lists.items():

    legends = [legend_titles[i] for i in requested_scenario_list]
    for output_type in ["median"]:

        dirpath = os.path.join(base_dir, output_type, foldername)
        os.makedirs(dirpath, exist_ok=True)

        for output_name in output_names:
            plot_outputs(output_type, output_name, requested_scenario_list, sc_linestyles, sc_colors, False, x_min=scenario_x_min, x_max=scenario_x_max
                        )# cumul_from=426)
            
            plt.legend(legends, fontsize=14)
            path = os.path.join(dirpath, f"{output_name}.png")
            plt.savefig(path)
            plt.close()

# Uncertainty around scenarios

In [ ]:
output_type = "uncertainty"
requested_scenario_list = requested_scenario_lists["all scenarios"]
for scenario in requested_scenario_list:
    scenarios_to_plot = [0, scenario] if scenario > 0 else [0]
    for output_name in output_names:
        plot_outputs(output_type, output_name, scenarios_to_plot, sc_linestyles, sc_colors, False, x_min=scenario_x_min, x_max=scenario_x_max)
        
        # add legend manually
        patches = [mpatches.Patch(color=COLORS[0][1], label=legend_titles[0])]
        if len(scenarios_to_plot) > 1: 
            patches.append(mpatches.Patch(color=COLORS[1][1], label=legend_titles[scenario]))
            
        plt.legend(handles=patches, title = "Model projections (median, 50% and 95% CI):")
        
        path = os.path.join(base_dir, output_type, f"{output_name}_scenario_{scenario}.png")
        plt.savefig(path)
        plt.close()

# Uncertainty around baseline only (Calibration plots)

In [ ]:
calibration_x_min, calibration_x_max = 720, 882

output_names = ["icu_occupancy", "hospital_occupancy", "notifications"]

if project.region_name == "national-capital-region":
    new_targets_path = os.path.join(PROJECTS_PATH, "sm_sir", "philippines", project.region_name, "new_targets.json")
    f = open(new_targets_path)
    new_targets = json.load(f)

for output_name in output_names:
    axis = plot_outputs("uncertainty", output_name, [0], sc_linestyles, sc_colors, False, x_min=calibration_x_min, x_max=calibration_x_max)  
    path = os.path.join(base_dir, 'calibration', f"{output_name}.png")
 

    if output_name.endswith("_occupancy") and project.region_name == "national-capital-region":
        values, times = new_targets[output_name]["values"], new_targets[output_name]["times"] 
        date_times = pd.to_datetime(times, origin="01Jan2020",unit="D")
    else:
        values, times = _get_target_values(project.plots, output_name)        
    
    _plot_targets_to_axis(axis, values, times, on_uncertainty_plot=True)

    plt.savefig(path)
    plt.close()



# Dump outputs to csv files

In [ ]:
csv_outputs = output_names
start_time = 700

includes_MLE = False
requested_quantiles = [0.025, 0.50, 0.975]

# for age in [str(int(5. * i)) for i in range(16)]:
#     csv_outputs.append(f"notificationsXagegroup_{age}")

def get_uncertainty_data(output_name, scenario_idx, quantile):
    mask = (
            (uncertainty_df["type"] == output_name)
            & (uncertainty_df["scenario"] == scenario_idx)
            & (uncertainty_df["quantile"] == quantile)
        )
    df = uncertainty_df[mask]
    times = df.time.unique()[1:]
    values = df["value"].tolist()[1:]
        
    return times, values

COVID_BASE_DATE = pd.datetime(2019, 12, 31)
start_date = pd.to_timedelta(start_time, unit="days") + (COVID_BASE_DATE)  

for scenario in scenario_list:
    df = pd.DataFrame()
    
    # include a column for the date
    t, _ = get_uncertainty_data("notifications", scenario, 0.5)
    df["date"] = pd.to_timedelta(t, unit="days") + (COVID_BASE_DATE)  
    
    for output in csv_outputs:
        if includes_MLE:
            derived_output_tables = db.load.load_derived_output_tables(calib_path, column=output)
            do_times, do_values = get_output_from_run_id(output, mcmc_tables, derived_output_tables, "MLE", scenario)            
            
            assert list(do_times[1:]) == list(t)
            do_values = list(do_values)[1:]        

            name = f"{output}_MLE"
            df[name] = do_values       
       
        if output in list(uncertainty_df["type"].unique()):
            for quantile in requested_quantiles:
                _, v = get_uncertainty_data(output, scenario, quantile)         
                name = f"{output}_{quantile}"
                df[name] = v            
    
    
    # trim the dataframe to keep requested times only
    df.drop(df[df.date < start_date].index, inplace=True)    
    
    path = os.path.join(base_dir, 'csv_files', f"outputs_scenario_{scenario}.csv")
    df.to_csv(path)
            
